In [41]:
import glob2
import os
from random import randint
from IPython.core.display import Image, display
import numpy as np
import numpy as np
import tensorflow as tf
from skimage import transform,io
import cv2

In [42]:
training_path = "/whhdata/person/MH_WHH_0153/measurements/1537860868501/rgb/"
#training_path = ""

glob_search_path = os.path.join(training_path, "**/*.jpg")
jpg_paths = glob2.glob(glob_search_path)
print("jpg_paths", len(jpg_paths))

jpg_paths 91


In [ ]:
index = randint(0,len(jpg_paths)-1)
img_filename = jpg_paths[index]
print("File %s: %s" % (index, img_filename))
#display(Image(filename=img_filename, width=257, height=353))

In [ ]:
img = cv2.imread(img_filename)
print(type(img))
print(img.shape)
img_rotate_90_clockwise = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
cv2.imwrite('90roation.jpg', img_rotate_90_clockwise)

In [ ]:
from PIL import Image
colorImage  = Image.open(img_filename)
transposed  = colorImage.transpose(Image.ROTATE_90)

In [ ]:
image = io.imread("90roation.jpg")

In [ ]:
training_path = ""

glob_search_path = os.path.join(training_path, "**/*.jpg")
jpg_paths = glob2.glob(glob_search_path)
print("jpg_paths", len(jpg_paths))

In [ ]:
index = randint(0,len(jpg_paths)-1)
img_filename = jpg_paths[index]
print("File %s: %s" % (index, img_filename))

In [ ]:
def imshow(img):
    import cv2
    import IPython
    _,ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)

In [ ]:

import PIL.Image
#from cStringIO import StringIO
import io
import IPython.display
import numpy as np
def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = io.StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

In [ ]:
PIL.Image.fromarray(image)

In [ ]:
import tensorflow as tf
import cv2
import time
import argparse
import os

import posenet

In [37]:
def main():

    with tf.Session() as sess:
        model_cfg, model_outputs = posenet.load_model(101, sess)
        output_stride = model_cfg['output_stride']


        start = time.time()
        input_image, draw_image, output_scale = posenet.read_imgfile(
            img_filename, scale_factor=1, output_stride=output_stride)

        heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = sess.run(
            model_outputs,
            feed_dict={'image:0': input_image}
        )

        pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multiple_poses(
            heatmaps_result.squeeze(axis=0),
            offsets_result.squeeze(axis=0),
            displacement_fwd_result.squeeze(axis=0),
            displacement_bwd_result.squeeze(axis=0),
            output_stride=output_stride,
            max_pose_detections=10,
            min_pose_score=0.25)

        keypoint_coords *= output_scale



        print()
        print("Results for image: %s" % img_filename)
        for pi in range(len(pose_scores)):
            if pose_scores[pi] == 0.:
                break
            print('Pose #%d, score = %f' % (pi, pose_scores[pi]))
            for ki, (s, c) in enumerate(zip(keypoint_scores[pi, :], keypoint_coords[pi, :, :])):
                print('Keypoint %s, score = %f, coord = %s' % (posenet.PART_NAMES[ki], s, c))

        print('Average FPS:', len(img_filename) / (time.time() - start))

In [38]:
if __name__ == "__main__":
    main()


Results for image: 90roation.jpg
Pose #0, score = 0.330989
Keypoint nose, score = 0.983822, coord = [406.6924836  522.03662495]
Keypoint leftEye, score = 0.998724, coord = [366.59928683 561.1453984 ]
Keypoint rightEye, score = 0.527671, coord = [367.06754511 468.70725946]
Keypoint leftEar, score = 0.417053, coord = [386.54850004 608.7030574 ]
Keypoint rightEar, score = 0.146144, coord = [404.68551421 392.64341221]
Keypoint leftShoulder, score = 0.331872, coord = [610.02861218 631.43431416]
Keypoint rightShoulder, score = 0.270064, coord = [623.89149576 352.52873402]
Keypoint leftElbow, score = 0.522320, coord = [842.30219144 703.64915178]
Keypoint rightElbow, score = 0.279837, coord = [851.81309269 305.13086491]
Keypoint leftWrist, score = 0.091696, coord = [1042.55851368  710.89471837]
Keypoint rightWrist, score = 0.475166, coord = [1031.75391559  254.35907524]
Keypoint leftHip, score = 0.217661, coord = [1052.32314349  598.05813864]
Keypoint rightHip, score = 0.113651, coord = [1039